# Building Production Patterns

**Goal:** Learn to build robust verification patterns that catch errors before they reach users.

**Time:** 20 minutes

**What You'll Build:**
- Retry logic for transient failures
- Fallback values for critical paths
- Alert system for verification failures
- Audit logging for compliance

---

## Setup

Import required libraries:

In [ ]:
from qwed_sdk import QWEDLocal
import logging
from datetime import datetime
from typing import Dict, Optional

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Pattern 1: Retry with Exponential Backoff

Handle transient failures (network issues, rate limits):

In [ ]:
import time

def verify_with_retry(client, query: str, max_retries: int = 3) -> Dict:
    """
    Retry verification with exponential backoff.
    
    Pattern: 1s, 2s, 4s delays between retries
    """
    for attempt in range(max_retries):
        try:
            result = client.verify_math(query)
            
            if result.verified:
                logger.info(f"✅ Verified on attempt {attempt + 1}")
                return result
            else:
                logger.warning(f"⚠️  Verification failed, attempt {attempt + 1}/{max_retries}")
                
        except Exception as e:
            logger.error(f"❌ Error on attempt {attempt + 1}: {e}")
            
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt  # Exponential backoff
                logger.info(f"Retrying in {wait_time}s...")
                time.sleep(wait_time)
            else:
                raise
    
    raise Exception("Verification failed after all retries")

# Test it
client = QWEDLocal(provider="openai", model="gpt-4o-mini")

result = verify_with_retry(client, "What is 2 + 2?")
print(f"Result: {result.value}")

## Pattern 2: Fallback Values

When verification fails, use safe defaults:

In [ ]:
def calculate_with_fallback(client, query: str, fallback_value: float) -> Dict:
    """
    Try verification, fall back to conservative value if it fails.
    
    Use case: Interest rate calculations where you'd rather
    underestimate than overestimate.
    """
    try:
        result = client.verify_math(query)
        
        if result.verified:
            return {
                "value": result.value,
                "source": "verified",
                "confidence": 100.0
            }
        else:
            logger.warning(f"⚠️  Verification failed, using fallback: {fallback_value}")
            return {
                "value": fallback_value,
                "source": "fallback",
                "confidence": 0.0,
                "reason": "verification_failed"
            }
            
    except Exception as e:
        logger.error(f"❌ Error: {e}, using fallback")
        return {
            "value": fallback_value,
            "source": "fallback",
            "confidence": 0.0,
            "reason": f"exception: {str(e)}"
        }

# Test with conservative fallback
result = calculate_with_fallback(
    client,
    "Calculate 10% interest on $1000",
    fallback_value=100.0  # Conservative estimate
)

print(f"Result: ${result['value']}")
print(f"Source: {result['source']}")
print(f"Confidence: {result['confidence']}%")

## Pattern 3: Alert System

Get notified when verifications fail:

In [ ]:
class VerificationPattern:
    """
    Production verification pattern with alerting.
    """
    
    def __init__(self, alert_threshold: int = 3):
        self.client = QWEDLocal(provider="openai", model="gpt-4o-mini")
        self.failure_count = 0
        self.alert_threshold = alert_threshold
        self.alerts = []
    
    def verify_and_alert(self, query: str) -> Dict:
        """Verify and send alert if failures exceed threshold."""
        try:
            result = self.client.verify_math(query)
            
            if result.verified:
                # Reset counter on success
                self.failure_count = 0
                return {"value": result.value, "verified": True}
            else:
                self.failure_count += 1
                
                if self.failure_count >= self.alert_threshold:
                    self._send_alert(f"High failure rate: {self.failure_count} consecutive failures")
                
                return {"value": None, "verified": False}
                
        except Exception as e:
            self.failure_count += 1
            self._send_alert(f"Exception: {e}")
            raise
    
    def _send_alert(self, message: str):
        """Send alert (in production: PagerDuty, Slack, email)."""
        alert = {
            "timestamp": datetime.now().isoformat(),
            "message": message,
            "failure_count": self.failure_count
        }
        self.alerts.append(alert)
        logger.error(f"🚨 ALERT: {message}")
        
        # In production, integrate with:
        # - PagerDuty: pagerduty_client.send_event(alert)
        # - Slack: slack_client.post_message(alert)
        # - Email: smtp.send(alert)

# Test alerting
verification pattern = VerificationPattern(alert_threshold=2)

result = verification pattern.verify_and_alert("What is 5 * 5?")
print(f"Verified: {result['verified']}")
print(f"Alerts sent: {len(verification pattern.alerts)}")

## Pattern 4: Audit Logging

Track every verification for compliance:

In [ ]:
import json

class AuditedVerifier:
    """Verifier with full audit trail."""
    
    def __init__(self, audit_file: str = "audit_log.jsonl"):
        self.client = QWEDLocal(provider="openai", model="gpt-4o-mini")
        self.audit_file = audit_file
    
    def verify_with_audit(self, query: str, context: Dict = None) -> Dict:
        """Verify and log to audit trail."""
        start_time = datetime.now()
        
        try:
            result = self.client.verify_math(query)
            
            audit_entry = {
                "timestamp": start_time.isoformat(),
                "query": query,
                "result": result.value if result.verified else None,
                "verified": result.verified,
                "confidence": result.confidence,
                "duration_ms": (datetime.now() - start_time).total_seconds() * 1000,
                "context": context or {}
            }
            
            self._write_audit(audit_entry)
            
            return result
            
        except Exception as e:
            audit_entry = {
                "timestamp": start_time.isoformat(),
                "query": query,
                "error": str(e),
                "verified": False,
                "context": context or {}
            }
            self._write_audit(audit_entry)
            raise
    
    def _write_audit(self, entry: Dict):
        """Write audit entry to JSONL file."""
        # In production: Write to database or centralized logging
        print(f"📋 Audit: {json.dumps(entry)}")

# Test auditing
audited_verifier = AuditedVerifier()

result = audited_verifier.verify_with_audit(
    "Calculate 15% of $200",
    context={"user_id": "user123", "session_id": "sess456"}
)

print(f"\nResult: ${result.value}")

## Exercise: Build Your Production verification pattern

Combine all patterns into one production-ready system:

In [ ]:
class ProductionVerifier:
    """
    TODO: Combine retry, fallback, alerting, and audit logging.
    
    Requirements:
    1. Retry up to 3 times with exponential backoff
    2. Fall back to conservative value if all retries fail
    3. Send alert after 5 consecutive failures
    4. Log every verification attempt
    """
    pass

# Test your implementation here

## Key Takeaways

✅ **Retry Logic:** Handle transient failures gracefully  
✅ **Fallback Values:** Always have a safe default  
✅ **Alert System:** Get notified when things go wrong  
✅ **Audit Logging:** Track everything for compliance  

**Next:** [Module 4: Advanced Patterns](../../module-4-advanced/README.md)